In [1]:
from glob import glob
from os import path
import seaborn as sns
import json
import os
import numpy as np
import pandas as pd
import re
from os.path import basename, splitext
import time
import pickle
import json

# from solver import Instance

from argparse import Namespace
import sys
sys.path.append('../../solver')
# from solver_output import practice_print
from solver_rostering import run_roster_solver_objval_w_baseline

In [2]:
# fixed params
RM = 1.5
GM = 0.8
h_min = 32
h_max = 48
max_n_diff = 3

refresh = False

In [3]:
list_city_dt = []

for file_in in os.listdir(r'../../instances'):
    base_file = splitext(basename(file_in))[0]
    base_file_split = base_file.split('_')
    file = base_file_split[0]+'_'+base_file_split[1]
    list_city_dt.append(file)

list_city_dt = list(set(list_city_dt))
list_city_dt.sort()
list_city_dt

list_city_dt = [name for name in list_city_dt if 'berlin' not in name]
list_city_dt

['frankfurt_db=0.50',
 'frankfurt_db=1.00',
 'frankfurt_db=2.00',
 'frankfurt_db=4.00',
 'lyon_db=0.50',
 'lyon_db=1.00',
 'lyon_db=2.00',
 'lyon_db=4.00',
 'paris_db=0.50',
 'paris_db=1.00',
 'paris_db=2.00',
 'paris_db=4.00']

In [4]:
def load_shift(shift_file, OC, RM, GM, model, max_n_shifts=None):
    with open(shift_file, 'r') as file:
        data = json.load(file)
        df_shifts = pd.DataFrame(data)
    df_shifts = df_shifts[(df_shifts['outsourcing_cost_multiplier']==OC)&(df_shifts['regional_multiplier']==RM)&(df_shifts['global_multiplier']==GM)]
    #fixed or flex
    if model in ['fixed','flex']:
        df_shifts = df_shifts[df_shifts['model']==model]
    #partflex
    else:
        df_shifts = df_shifts[(df_shifts['model']==model)&(df_shifts['max_n_shifts']==max_n_shifts)]
    df_shifts.reset_index(drop = True, inplace=True)
    df_shifts = df_shifts[['region','shifts_start','shifts_end']]
    dict_shifts = df_shifts.set_index('region').to_dict(orient='index')
    return dict_shifts

In [5]:
count = 0
matching_shifts_1 = 0
matching_shifts_2 = 0

for base_file in list_city_dt:


    weekday_ = f'{base_file}_dt=doublepeak.json'
    weekend_ = f'{base_file}_dt=uniform.json'
    shift_file_weekday = f"../../shifts/{weekday_}"
    shift_file_weekend = f"../../shifts/{weekend_}"


    for OC in [1.2, 1.5, 1.8, 2.0, 2.5]:

        count +=1

        # Group 1
        shift_weekend_flex = load_shift(shift_file_weekend, OC, RM, GM, "flex")
        shift_weekday_flex = load_shift(shift_file_weekday, OC, RM, GM, "flex")
       
        shift_weekend_partflex_4 = load_shift(shift_file_weekend, OC, RM, GM, "partflex", 4)
        shift_weekday_partflex_4 = load_shift(shift_file_weekday, OC, RM, GM, "partflex", 4)

        shift_weekend_partflex_3 = load_shift(shift_file_weekend, OC, RM, GM, "partflex", 3)
        shift_weekday_partflex_3 = load_shift(shift_file_weekday, OC, RM, GM, "partflex", 3)


        if shift_weekend_flex == shift_weekend_partflex_4 and \
            shift_weekend_partflex_4 == shift_weekend_partflex_3 and \
            shift_weekday_flex == shift_weekday_partflex_4 and \
            shift_weekday_partflex_4 == shift_weekday_partflex_3:
            matching_shifts_1 += 1
        else:
            print(OC, base_file, "shifts don't match G1")

        # Group 2
        shift_weekend_fixed = load_shift(shift_file_weekend, OC, RM, GM, "fixed")
        shift_weekday_fixed = load_shift(shift_file_weekday, OC, RM, GM, "fixed")
       
        shift_weekend_partflex_2 = load_shift(shift_file_weekend, OC, RM, GM, "partflex", 2)
        shift_weekday_partflex_2 = load_shift(shift_file_weekday, OC, RM, GM, "partflex", 2)   

        if    shift_weekend_fixed == shift_weekend_partflex_2 and \
            shift_weekday_fixed == shift_weekday_partflex_2:
            matching_shifts_2 += 1
        else:
            print(OC, base_file, "shifts don't match G2")
            print(shift_weekend_fixed)
            print(shift_weekend_partflex_2)




1.2 frankfurt_db=0.50 shifts don't match G2
{0: {'shifts_start': {'0': 0, '1': 4}, 'shifts_end': {'0': 4, '1': 8}}, 1: {'shifts_start': {'0': 0, '1': 4}, 'shifts_end': {'0': 4, '1': 8}}, 2: {'shifts_start': {'0': 0, '1': 4}, 'shifts_end': {'0': 4, '1': 8}}, 3: {'shifts_start': {'0': 0, '1': 4}, 'shifts_end': {'0': 4, '1': 8}}}
{0: {'shifts_start': {'0': 0, '1': 4}, 'shifts_end': {'0': 4, '1': 8}}, 1: {'shifts_start': {'0': 0, '1': 4}, 'shifts_end': {'0': 4, '1': 8}}, 2: {'shifts_start': {'0': 0, '1': 4}, 'shifts_end': {'0': 4, '1': 8}}, 3: {'shifts_start': {'0': 0, '1': 4}, 'shifts_end': {'0': 4, '1': 8}}}
1.5 frankfurt_db=0.50 shifts don't match G2
{0: {'shifts_start': {'0': 0, '1': 4}, 'shifts_end': {'0': 4, '1': 8}}, 1: {'shifts_start': {'0': 0, '1': 4}, 'shifts_end': {'0': 4, '1': 8}}, 2: {'shifts_start': {'0': 0, '1': 4}, 'shifts_end': {'0': 4, '1': 8}}, 3: {'shifts_start': {'0': 0, '1': 4}, 'shifts_end': {'0': 4, '1': 8}}}
{0: {'shifts_start': {'0': 0, '1': 4}, 'shifts_end': {'0'

In [6]:
print(matching_shifts_1/count)

0.9833333333333333


In [7]:
print(matching_shifts_1/count)

0.9833333333333333
